# Generate All Change Maps (Yield and Variability)

**Goal:** This notebook automates the creation of change maps for every crop dataset. For each dataset, it generates and saves two maps:
1.  **Change in Average Yield:** Comparing the last decade (2007-2016) to the first (1981-1990).
2.  **Change in Yield Variability:** Comparing the second half of the period (1999-2016) to the first (1981-1998).

All files are saved into a structured folder system within `reports/figures/change_maps/`.

In [1]:
# Cell 1: The Complete Change Map Generation Script
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import os
import glob
import re

def generate_change_maps(yield_data: xr.DataArray, crop_name: str, output_dir: str):
    """
    Generates and saves the 'change in average' and 'change in variability' maps for a given yield dataset.
    """
    print(f"--- Generating change maps for: {crop_name.upper()} ---")
    os.makedirs(output_dir, exist_ok=True)

    # --- 1. Map of Change in Average Yield ---
    early_period = slice('1981', '1990')
    late_period = slice('2007', '2016')
    yield_early = yield_data.sel(time=early_period).mean(dim='time')
    yield_late = yield_data.sel(time=late_period).mean(dim='time')
    percent_change = xr.where(yield_early > 0.1, 100 * (yield_late - yield_early) / yield_early, np.nan)

    fig, ax = plt.subplots(figsize=(15, 8), subplot_kw={'projection': ccrs.PlateCarree()})
    ax.add_feature(cfeature.LAND, facecolor='#f0f0f0')
    ax.coastlines(linewidth=0.5)
    ax.add_feature(cfeature.BORDERS, linestyle=':', edgecolor='gray', linewidth=0.5)
    percent_change.plot(ax=ax, cmap='RdBu_r', transform=ccrs.PlateCarree(), vmax=100, center=0, cbar_kwargs={'label': 'Percent Change in Average Yield (%)'})
    ax.set_title(f'Change in Average Yield: {crop_name} (2007-16 vs 1981-90)')
    
    avg_change_path = os.path.join(output_dir, f'change_avg_yield_{crop_name}.png')
    plt.savefig(avg_change_path, dpi=300, bbox_inches='tight')
    plt.close(fig)
    print(f"Saved average change map to {avg_change_path}")

    # --- 2. Map of Change in Yield Variability ---
    first_half = slice('1981', '1998')
    second_half = slice('1999', '2016')
    variability_first = yield_data.sel(time=first_half).std(dim='time')
    variability_second = yield_data.sel(time=second_half).std(dim='time')
    percent_change_variability = xr.where(variability_first > 0.1, 100 * (variability_second - variability_first) / variability_first, np.nan)

    fig, ax = plt.subplots(figsize=(15, 8), subplot_kw={'projection': ccrs.PlateCarree()})
    ax.add_feature(cfeature.LAND, facecolor='#f0f0f0')
    ax.coastlines(linewidth=0.5)
    ax.add_feature(cfeature.BORDERS, linestyle=':', edgecolor='gray', linewidth=0.5)
    percent_change_variability.plot(ax=ax, cmap='RdBu_r', transform=ccrs.PlateCarree(), vmax=100, center=0, cbar_kwargs={'label': 'Percent Change in Yield Variability (%)'})
    ax.set_title(f'Change in Yield Variability: {crop_name} (1999-2016 vs 1981-1998)')

    var_change_path = os.path.join(output_dir, f'change_variability_{crop_name}.png')
    plt.savefig(var_change_path, dpi=300, bbox_inches='tight')
    plt.close(fig)
    print(f"Saved variability change map to {var_change_path}")
    print("-" * 50)


# --- Main Execution ---
BASE_DATA_DIR = '../data/'
BASE_OUTPUT_DIR = '../reports/figures/change_maps/'

# --- Part A: Process all individual crop folders ---
crop_folders = [d for d in os.listdir(BASE_DATA_DIR) if os.path.isdir(os.path.join(BASE_DATA_DIR, d))]
for crop_name in crop_folders:
    data_path = os.path.join(BASE_DATA_DIR, crop_name, 'yield_*.nc4')
    with xr.open_mfdataset(data_path, combine='nested', concat_dim='time') as ds:
        years = range(1981, 2017)
        ds = ds.assign_coords(time=years)
        yield_data = ds['var'].compute() # .compute() loads it all into memory
        
        output_path = os.path.join(BASE_OUTPUT_DIR, crop_name)
        generate_change_maps(yield_data, crop_name, output_path)

# --- Part B: Process the Combined Total Yield ---
CROPS_TO_COMBINE = ['maize', 'rice', 'wheat', 'soybean']
all_crop_dataarrays = []
for crop in CROPS_TO_COMBINE:
    data_path = os.path.join(BASE_DATA_DIR, crop, 'yield_*.nc4')
    with xr.open_mfdataset(data_path, combine='nested', concat_dim='time') as ds:
        all_crop_dataarrays.append(ds['var'])

if all_crop_dataarrays:
    combined_yield = xr.concat(all_crop_dataarrays, dim='crop').fillna(0).sum(dim='crop').compute()
    years = range(1981, 2017)
    combined_yield = combined_yield.assign_coords(time=years)
    
    output_path = os.path.join(BASE_OUTPUT_DIR, 'combined_total_yield')
    generate_change_maps(combined_yield, 'combined_total_yield', output_path)

print("--- All Done! ---")

--- Generating change maps for: MAIZE ---
Saved average change map to ../reports/figures/change_maps/maize\change_avg_yield_maize.png


C:\ProgramData\miniconda3\envs\climarisc\lib\site-packages\numpy\lib\_nanfunctions_impl.py:2019: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\ProgramData\miniconda3\envs\climarisc\lib\site-packages\numpy\lib\_nanfunctions_impl.py:2019: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


Saved variability change map to ../reports/figures/change_maps/maize\change_variability_maize.png
--------------------------------------------------
--- Generating change maps for: MAIZE_MAJOR ---
Saved average change map to ../reports/figures/change_maps/maize_major\change_avg_yield_maize_major.png


C:\ProgramData\miniconda3\envs\climarisc\lib\site-packages\numpy\lib\_nanfunctions_impl.py:2019: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\ProgramData\miniconda3\envs\climarisc\lib\site-packages\numpy\lib\_nanfunctions_impl.py:2019: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


Saved variability change map to ../reports/figures/change_maps/maize_major\change_variability_maize_major.png
--------------------------------------------------
--- Generating change maps for: MAIZE_SECOND ---
Saved average change map to ../reports/figures/change_maps/maize_second\change_avg_yield_maize_second.png


...

Saved variability change map to ../reports/figures/change_maps/wheat\change_variability_wheat.png
--------------------------------------------------
--- Generating change maps for: WHEAT_SPRING ---
Saved average change map to ../reports/figures/change_maps/wheat_spring\change_avg_yield_wheat_spring.png


C:\ProgramData\miniconda3\envs\climarisc\lib\site-packages\numpy\lib\_nanfunctions_impl.py:2019: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\ProgramData\miniconda3\envs\climarisc\lib\site-packages\numpy\lib\_nanfunctions_impl.py:2019: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


Saved variability change map to ../reports/figures/change_maps/wheat_spring\change_variability_wheat_spring.png
--------------------------------------------------
--- Generating change maps for: WHEAT_WINTER ---
Saved average change map to ../reports/figures/change_maps/wheat_winter\change_avg_yield_wheat_winter.png


C:\ProgramData\miniconda3\envs\climarisc\lib\site-packages\numpy\lib\_nanfunctions_impl.py:2019: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\ProgramData\miniconda3\envs\climarisc\lib\site-packages\numpy\lib\_nanfunctions_impl.py:2019: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


Saved variability change map to ../reports/figures/change_maps/wheat_winter\change_variability_wheat_winter.png
--------------------------------------------------
--- Generating change maps for: COMBINED_TOTAL_YIELD ---
Saved average change map to ../reports/figures/change_maps/combined_total_yield\change_avg_yield_combined_total_yield.png
Saved variability change map to ../reports/figures/change_maps/combined_total_yield\change_variability_combined_total_yield.png
--------------------------------------------------
--- All Done! ---
